In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils  import plot_model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Add
import tensorflow as tf

In [ ]:
# Filters in Series Array
# x=previous_layer, f_in=4, f_out=16

def FSA(x, f_in, f_out): 
    layers = []
    for i in [1,3,5]:
        conv = Conv2D(f_in, (1,1), padding='same')(x)
        for j in range(i):
            conv = Conv2D(f_out, (1,3), padding='same')(conv)
            conv = Conv2D(f_out, (3,1), padding='same')(conv)
        
        layers.append(conv)
    
    out = Concatenate()(layers)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out




# x=previous_layer, n=5, f_in=4, f_out=16

def Single_Kernel_Series_Block(x, n, f_in, f_out):  
  layer_list = []
  for i in range(n):
      fsa_layer = FSA(x, f_in, f_out)
      layer_list.append(fsa_layer)
           
  layer_concate = Concatenate()(layer_list)
    
  x = Conv2D(layer_concate.shape[-1], (1,1), padding='same', activation='relu')(x)
  out      = Add()([x, layer_concate])
    
  out = BatchNormalization()(out)
  out = Activation('relu')(out)
  return out

In [ ]:
# x=previous_layer, n=5, f_in=4, f_mid=16, f_out=4

def Single_Kernel_Parallel_Block(x, n, f_in, f_mid, f_out):
    layer_list = []
    for i in range(n):
        conv  = Conv2D(f_in, (1,1), padding='same')(x)
        
        conv1 = Conv2D(f_mid, (1,3), padding='same')(conv)
        conv2 = Conv2D(f_mid, (3,1), padding='same')(conv)
        
        conv  = Concatenate()([conv1, conv2])
        conv  = Conv2D(f_out, (1,1), padding='same')(conv)
        layer_list.append(conv)
        
    layer_concate = Concatenate()(layer_list)
    
    x = Conv2D(layer_concate.shape[-1], (1,1), padding='same')(x)
    out      = Add()([x, layer_concate])
    
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out

In [ ]:
#Parallel Out Array
# x=previous_layer, f_in=4, f_out=16

def POA(x, f_in, f_out):    
    layers = []
    for i in [1,3,5]:
        conv = Conv2D(f_in, (1,1), padding='same')(x)
        for j in range(i):
            conv2 = Conv2D(f_out, (1,3), padding='same')(conv)
            layers.append(conv2)
            conv2 = Conv2D(f_out, (3,1), padding='same')(conv)
            layers.append(conv2)
    
    out = Concatenate()(layers)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out





# x=previous_layer, n=5, f_in=4, f_out=16

def Single_Kernel_Expand_Block(x, n, f_in, f_out):    
  layer_list = []
  for i in range(n):
      poa_layer = POA(x, f_in, f_out)
      layer_list.append(poa_layer)
           
  layer_concate = Concatenate()(layer_list)
    
  x = Conv2D(layer_concate.shape[-1], (1,1), padding='same', activation='relu')(x)
  out      = Add()([x, layer_concate])
    
  out = BatchNormalization()(out)
  out = Activation('relu')(out)
  return out

In [ ]:
# n size Filters in Parallel
# x=previous_layer, f_in=4, f_out=16, n=3/5/7

def nFP(x, n, f_in, f_out):                            
    conv = Conv2D(f_in, (1,1), padding='same', activation='relu')(x)
    
    conv1 = Conv2D(f_out, (1,n), padding='same', activation='relu')(conv)
    conv2 = Conv2D(f_out, (n,1), padding='same', activation='relu')(conv)
    
    out  = Concatenate()([conv1, conv2])
    out  = Activation('relu')(conv)
    return out





# x=previous_layer, N=5, f_in=4, f_out=16

def n_Kernel_Parallel_Block(x, N, f_in, f_out):   
    layer_list = []
    for i in range(N):
        layer = nFP(x, 3, f_in, f_out)
        layer_list.append(layer)
        layer = nFP(x, 5, f_in, f_out)
        layer_list.append(layer)
        layer = nFP(x, 7, f_in, f_out)
        layer_list.append(layer)
           
    layer_concate = Concatenate()(layer_list)
    
    x = Conv2D(layer_concate.shape[-1], (1,1), padding='same', activation='relu')(x)
    out      = Add()([x, layer_concate])
    
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out

In [ ]:
# Factorized Kernel Parallel Out
# x=previous_layer, f_in=4, f_out=16, n=3/5/7

def FPO(x, n, f_in, f_mid, f_out):                                     
    conv = Conv2D(f_in, (1,1), padding='same', activation='relu')(x)
    if n==3:
        l=1
        filters = [f_out]
    elif n==5:
        l=2
        filters = [f_mid, f_out]
    else:
        l=3
        filters = [f_mid, f_mid, f_out]
        
    for i in range(l):
        conv = Conv2D(filters[i] , (3,3), padding='same', activation='relu')(conv)
        
    return conv




# n Kernel Expand Block >>> (Block name in the Paper)
# x=previous_layer, N=5, f_in=4, f_out=16

def n_Kernel_Expand_Block(x, N, f_in, f_mid, f_out):     
    layer_list = []
    for i in range(N):
        layer = FPO(x, 3, f_in, f_mid, f_out)
        layer_list.append(layer)
        layer = FPO(x, 5, f_in, f_mid, f_out)
        layer_list.append(layer)
        layer = FPO(x, 7, f_in, f_mid, f_out)
        layer_list.append(layer)
        
    layer_concate = Concatenate()(layer_list)
    
    x   = Conv2D(layer_concate.shape[-1], (1,1), padding='same', activation='relu')(x)
    out = Add()([x, layer_concate])

    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out

In [ ]:
def pipeLine(x):
    x1 = Conv2D(32, (3,3), padding='same', activation='relu')(x)
    x1 = MaxPooling2D((3,3), strides=(1,1), padding='same')(x1)

    x1 = Conv2D(64, (3,3), padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((3,3), strides=(1,1), padding='same')(x1)

    x1 = Conv2D(96, (3,3), padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((3,3), strides=(1,1), padding='same')(x1)
    return x1
    
    
def FullyConnected(x):
    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(120, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(120, activation='relu')(x)
    x = Dense(10, activation='softmax')(x)
    return x

In [ ]:
def PES(x):
  x1 = Single_Kernel_Parallel_Block(x, 5, 4, 16, 4)
  x2 = n_Kernel_Parallel_Block(x, 5, 4, 16)
  x3 = pipeLine(x)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)


  x1 = Single_Kernel_Expand_Block(x4, 5, 4, 16)
  x2 = n_Kernel_Expand_Block(x4, 5, 4, 16, 4)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])


  x1 = Single_Kernel_Series_Block(x4, 5, 4, 16)
  x2 = n_Kernel_Parallel_Block(x4, 5, 4, 16)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)

  return x4

In [ ]:
def PSE(x):
  x1 = Single_Kernel_Parallel_Block(x, 5, 4, 16, 4)
  x2 = n_Kernel_Parallel_Block(x, 5, 4, 16)
  x3 = pipeLine(x)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)


  x1 = Single_Kernel_Series_Block(x4, 5, 4, 16)
  x2 = n_Kernel_Parallel_Block(x4, 5, 4, 16)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])


  x1 = Single_Kernel_Expand_Block(x4, 5, 4, 16)
  x2 = n_Kernel_Expand_Block(x4, 5, 4, 16, 4)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)

  return x4

In [ ]:
def SEP(x):
  x1 = Single_Kernel_Series_Block(x, 5, 4, 16)
  x2 = n_Kernel_Parallel_Block(x, 5, 4, 16)
  x3 = pipeLine(x)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)


  x1 = Single_Kernel_Expand_Block(x4, 5, 4, 16)
  x2 = n_Kernel_Expand_Block(x4, 5, 4, 16, 4)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])


  x1 = Single_Kernel_Parallel_Block(x4, 5, 4, 16, 4)
  x2 = n_Kernel_Parallel_Block(x4, 5, 4, 16)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)

  return x4

In [ ]:
def SPE(x):
  x1 = Single_Kernel_Series_Block(x, 15, 4, 16)
  x2 = n_Kernel_Parallel_Block(x, 15, 4, 16)
  x3 = pipeLine(x)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)


  x1 = Single_Kernel_Parallel_Block(x4, 15, 4, 16, 4)
  x2 = n_Kernel_Parallel_Block(x4, 15, 4, 16)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])


  x1 = Single_Kernel_Expand_Block(x4, 15, 4, 16)
  x2 = n_Kernel_Expand_Block(x4, 15, 4, 16, 4)
  x3 = pipeLine(x4)
  xC = Concatenate()([x1,x2,x3])
  res1  = Conv2D(xC.shape[-1], (1,1), padding='same', activation='relu')(x4)
  x4    = Add()([xC, res1])
  x4    = MaxPooling2D(3,3)(x4)

  return x4

In [ ]:
img = Input( shape=(70,70,1) )

x = Conv2D(8, (3,3), padding='same', activation='relu')(img)
x = MaxPooling2D(2,2)(x)
x = Conv2D(4, (3,3), padding='same', activation='relu')(x)
x = MaxPooling2D(2,2)(x)

x = PSE(x)
out   = FullyConnected(x)
model = Model(inputs=img, outputs=out)